In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from  helper_functions import *

Fetch data sets

In [57]:
#House sale price and other sale details downloaded from https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads
#Column data from https://www.gov.uk/guidance/about-the-price-paid-data#explanations-of-column-headers-in-the-ppd
#2024 data
house_2024 = pd.read_csv('http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2024.csv')
house_2024.columns = ['Ref_no','Price','Date','Postcode','Type','New','Tenure','Address_1','Address_2','Street','Locality','City','District','County','PPD_category','Record_Status']
#2023 data
house_2023= pd.read_csv('http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2023.csv')
house_2023.columns = ['Ref_no','Price','Date','Postcode','Type','New','Tenure','Address_1','Address_2','Street','Locality','City','District','County','PPD_category','Record_Status']
#2022 data
house_2022 = pd.read_csv('http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2022.csv')
house_2022.columns = ['Ref_no','Price','Date','Postcode','Type','New','Tenure','Address_1','Address_2','Street','Locality','City','District','County','PPD_category','Record_Status']

#files linking lsoa, msoa and ltla codes to postcodes
#Needed to join cencus data to postcode
#Files from https://geoportal.statistics.gov.uk/datasets/c4f84c38814d4b82aa4760ade686c3cc/about
lsoa_msoa_raw = pd.read_csv('https://www.arcgis.com/sharing/rest/content/items/c4f84c38814d4b82aa4760ade686c3cc/data',compression='zip',dtype=str)
#From https://geoportal.statistics.gov.uk/datasets/c4f84c38814d4b82aa4760ade686c3cc/about
ltla_raw = pd.read_csv('https://www.arcgis.com/sharing/rest/content/items/bc8f6d1f6ee64111b6a59b22c6605f3b/data',compression='zip',dtype=str)

#Latitude and longitude data for each postcode
#postcode location from file downlaoded from https://www.freemaptools.com/download-uk-postcode-lat-lng.htm
postcode_lat_lon_raw = pd.read_csv('https://data.freemaptools.com/download/full-uk-postcodes/ukpostcodes.zip',compression='zip',dtype=str)

#Demographics data
#downloaded from https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/lowersuperoutputareamidyearpopulationestimates
#preprocessed in excel and saved as csv
demographics = pd.read_csv('input_files/demographics.csv')

#Average Commute 
#Downloaded from https://www.ons.gov.uk/datasets/TS058/editions/2021/versions/4/filter-outputs/1242b10f-061d-4db7-9e69-ab1f2036e00f#get-data
#preprocessed in excel and saved as csv
commute = pd.read_csv('input_files/commute.csv')

#2023 wage data downloaded from https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/smallareaincomeestimatesformiddlelayersuperoutputareasenglandandwales
#excel converted to csv
wages = pd.read_csv('input_files/wages.csv')

#rural/urban
#excel file from https://www.ons.gov.uk/methodology/geography/geographicalproducts/ruralurbanclassifications/2021ruralurbanclassification
#table 1b converted to csv using excel
rural_urban = pd.read_csv('input_files/rural_urban.csv')

In [14]:
train_frames = [house_2022, house_2023, house_2024]
train_house = pd.concat(train_frames)
train_house.drop(['Ref_no','PPD_category','Record_Status'], axis=1, inplace=True)
train_house.reset_index(drop=True, inplace=True)
train_house.to_csv('train_house.csv', index=False)

A: Fill missing postcodes

In [27]:
print(f'{sum(train_house['Postcode'].isnull())} missing postcodes in train_house')

7794 missing postcodes in train_house


In [ ]:
train_house_1 = train_house.copy()
fill_empty_postcode(train_house_1, train_house)
train_house_1.to_csv('train_house_1.csv', index=False)

#Takes a long time to run. Progress will be displayed

In [29]:
print(f'{sum(train_house_1['Postcode'].isnull())} missing postcodes in train_house_1')

0 missing postcodes in train_house_1


B: Sample unique postcodes

In [83]:
postcodes = train_house_1['Postcode'].unique()

train_postcode_1 = pd.DataFrame({'postcode':postcodes})

C: Join LTLA, LSOA, MSOA, Latitude and Longitude on postcode

In [5]:
train_postcode_2 = train_postcode_1.copy()

lsoa_msoa = lsoa_msoa_raw[['pcds', 'lsoa21cd', 'msoa21cd']].set_index('pcds')
ltla = ltla_raw[['pcds', 'ltla22cd']].set_index('pcds')
lat_lon = postcode_lat_lon_raw.drop('id', axis=1).set_index('postcode')

train_postcode_2 = train_postcode_2.join(lsoa_msoa, on='postcode')
train_postcode_2 = train_postcode_2.join(ltla, on='postcode')
train_postcode_2 = train_postcode_2.join(lat_lon, on='postcode')

train_postcode_2.rename({'lsoa21cd':'lsoa', 'msoa21cd':'msoa', 'ltla22cd':'ltla'}, axis=1, inplace=True)

D: Fill missing LTLA, LSOA, MSOA, Latitude and Longitude

In [7]:
print(f'{sum(train_postcode_2['lsoa'].isna())} missing lsoa')
print(f'{sum(train_postcode_2['msoa'].isna())} missing msoa')
print(f'{sum(train_postcode_2['ltla'].isna())} missing ltla')
print(f'{sum(train_postcode_2['latitude'].isna())} missing latitude')
print(f'{sum(train_postcode_2['longitude'].isna())} missing longitude')

86 missing lsoa
86 missing msoa
4294 missing ltla
2251 missing latitude
2251 missing longitude


In [ ]:
train_postcode_3 = train_postcode_2.copy()

fill_missing_ltla(train_postcode_3, ltla_raw)
fill_missing_lsoa(train_postcode_3, lsoa_msoa_raw)
fill_missing_lat_lon(train_postcode_3, postcode_lat_lon_raw)

train_postcode_3.to_csv('train_postcode_3.csv', index=False)

#Takes time to run. Will print progress to console.

In [18]:
print(f'{sum(train_postcode_3['lsoa'].isna())} missing lsoa')
print(f'{sum(train_postcode_3['msoa'].isna())} missing msoa')
print(f'{sum(train_postcode_3['ltla'].isna())} missing ltla')
print(f'{sum(train_postcode_3['latitude'].isna())} missing latitude')
print(f'{sum(train_postcode_3['longitude'].isna())} missing longitude')

0 missing lsoa
0 missing msoa
0 missing ltla
0 missing latitude
0 missing longitude


E: Join additional postcode data

In [77]:
train_postcode_3 = pd.read_csv('train_postcode_3.csv')
train_postcode_4 = train_postcode_3.copy()
train_postcode_4 = train_postcode_4.join(demographics.set_index('LSOA 2021 Code'), on='lsoa')
train_postcode_4 = train_postcode_4.join(commute.set_index('LTLA'), on='ltla')
train_postcode_4 = train_postcode_4.join(wages.set_index('msoa'), on='msoa')

F: Calculate additional metrics

In [ ]:
train_postcode_5 = train_postcode_4.copy()

#England or wales
#Determine using first letter of LTLA code
train_postcode_5['Eng_Wal'] = train_postcode_5['ltla'].str[0]
train_postcode_5['Eng_Wal'] = train_postcode_5['Eng_Wal'].map({'E':'England','W':'Wales'})

#Distance from London
#getting the distance from the centre of london (chosen to be Big Ben)
#This block takes a short time to run, progress will be printed
train_postcode_5['London_distance'] = london_distance(train_postcode_5['latitude'], train_postcode_5['longitude'])

G: Join postcode data to main df

In [33]:
train_house_2 = train_house_1.join(train_postcode_5.set_index('postcode'), on='Postcode')
train_house_2.to_csv('train_house_2.csv', index=False)

H: Calculate helper metrics for both train and test frames (training data only)

In [99]:
train_house_2 = pd.read_csv('train_house_2.csv')

#Earliest date in the training data
earliest_date = pd.DataFrame({'earliest_date':pd.to_datetime([train_house_2['Date'].min()])})
earliest_date.to_csv('helper_metrics/train_earliest_date.csv', index=False)

#Number of listings for each lsoa code, to calculate per capita listings
lsoa_count = pd.DataFrame(train_house_2['lsoa'].value_counts())
counts = train_postcode_5[['lsoa','lsoa population']].drop_duplicates()
lsoa_count = lsoa_count.join(counts.set_index('lsoa'))
lsoa_count['listings_per_capita'] = lsoa_count['count']/lsoa_count['lsoa population']
lsoa_count.drop(['count','lsoa population'], axis=1, inplace=True)
lsoa_count = lsoa_count.round(4)
lsoa_count.to_csv('helper_metrics/train_lsoa_count.csv', index=True)

I: Calculate additional metrics

In [165]:
train_house_2 = pd.read_csv('train_house_2.csv')

#per capita listings by lsoa area
lsoa_count = pd.read_csv('helper_metrics/train_lsoa_count.csv')
train_house_3 = train_house_2.join(lsoa_count.set_index('lsoa'), on='lsoa')

#days since first listing in train df
first_date_df = pd.read_csv('helper_metrics/train_earliest_date.csv')
first_date = pd.to_datetime(first_date_df.iloc[0,0])
days_since_first = (pd.to_datetime(train_house_3['Date']) - first_date).dt.days
train_house_3['days_since_first'] = days_since_first

#Day of hte year encoded as sine and cosine wave
day_of_year = pd.to_datetime(train_house_3['Date']).dt.dayofyear
train_house_3['sine_day'] = np.sin(day_of_year*2*np.pi/365)
train_house_3['cosine_day'] = np.cos(day_of_year*2*np.pi/365)

J: Remove unneeded columns

In [150]:
train_house_4 = train_house_3.drop(['Postcode', 'Date', 'Address_1', 'Address_2', 'Street', 'Locality', 'City', 'District',
                                    'County', 'msoa', 'lsoa', 'ltla', 'lsoa population'], axis=1)

K: Investigate and fix missing values

In [158]:
train_house_4.info(show_counts=True)
#one item seems to be missing postcode related data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2832895 entries, 0 to 2832894
Data columns (total 17 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   Price                2832895 non-null  int64  
 1   Type                 2832895 non-null  object 
 2   New                  2832895 non-null  object 
 3   Tenure               2832895 non-null  object 
 4   latitude             2832895 non-null  float64
 5   longitude            2832895 non-null  float64
 6   gender_ratio         2832894 non-null  float64
 7   child_ratio          2832894 non-null  float64
 8   elderly_ratio        2832894 non-null  float64
 9   commute_distance     2832894 non-null  float64
 10  Income               2832894 non-null  float64
 11  Eng_Wal              2832894 non-null  object 
 12  London_distance      2832895 non-null  float64
 13  listings_per_capita  2832894 non-null  float64
 14  days_since_first     2832895 non-null  int64  
 15

In [159]:
train_house_4[train_house_4['gender_ratio'].isna()]
#Latitude/longitude (56.208453,-3.200442) is in scotland. 
#Looks like this was included in the england/Wales data by accident
#Should be removed

,Price,Type,New,Tenure,latitude,longitude,gender_ratio,child_ratio,elderly_ratio,commute_distance,Income,Eng_Wal,London_distance,listings_per_capita,days_since_first,sine_day,cosine_day
1820838,490000,F,N,L,56.208453,-3.200442,NaN,NaN,NaN,NaN,NaN,NaN,560.875208,NaN,709,-0.337523,0.941317


In [163]:
train_house_final = train_house_4.dropna()
train_house_final.to_csv('train_house_final.csv', index=False)